In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install torchvision

In [ ]:
pip install opencv-python

In [ ]:
pip install scikit-image

In [ ]:
pip install numpy

In [ ]:
pip install pillow

In [1]:
#from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import cv2
from collections import defaultdict
import os
from skimage import io, transform
import random
import pickle


C:\Users\aiden\AppData\Local\Temp\ipykernel_26156\2121635723.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
'''with open('/content/drive/MyDrive/CTdata/dataloader.pkl', 'rb') as file:

    CT_dataset = pickle.load(file)

with open('/content/drive/MyDrive/CTdata/meanstd.pkl', 'wb') as file:

    mean,std = pickle.load(file)'''


"with open('/content/drive/MyDrive/CTdata/dataloader.pkl', 'rb') as file:\n\n    CT_dataset = pickle.load(file)\n\nwith open('/content/drive/MyDrive/CTdata/meanstd.pkl', 'wb') as file:\n\n    mean,std = pickle.load(file)"

In [3]:
datapath = "G:/My Drive/CTdata/curated_data/curated_data"
dir = "G:/My Drive/CTdata/"

In [4]:
covid = pd.read_csv("G:/My Drive/CTdata/meta_data_covid.csv", encoding = "latin_1")
normal = pd.read_csv("G:/My Drive/CTdata/meta_data_normal.csv")
cap = pd.read_csv("G:/My Drive/CTdata/meta_data_cap.csv")

In [5]:
def show_scan(idx):
  img = np.asarray(Image.open(datapath+'/1NonCOVID/' + normal.iloc[idx]["File name"]))
  imgplot = plt.imshow(img)

In [6]:
def read_scan(idx):
  img = np.asarray(Image.open(datapath+'/1NonCOVID/' + normal.iloc[idx]["File name"]))
  imgplot = plt.imread(img)

In [7]:
'''with open('/content/drive/MyDrive/CTdata/all_images.pkl', 'rb') as file:

    all_images = pickle.load(file)'''

"with open('/content/drive/MyDrive/CTdata/all_images.pkl', 'rb') as file:\n\n    all_images = pickle.load(file)"

In [8]:
def is_grayscale(image):
  """
  Checks if an image is grayscale.
  Args:
    image: The image to check.
  Returns:
    True if the image is grayscale, False otherwise.
  """
  # Convert the image to HSV color space.
  hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  # Get the saturation channel of the image.
  saturation_channel = np.array(hsv_image[:, :, 1])
  # Check if all pixels in the saturation channel are 0.
  return not np.count_nonzero(saturation_channel)


In [9]:
class CTScanDataset(Dataset):

    def __init__(self, root_dir, data=None, patientid_to_label = None, invalid_patients = None, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.normal = pd.read_csv(dir + "meta_data_normal.csv")
        self.covid = pd.read_csv(dir + "meta_data_covid.csv", encoding = "latin_1")
        self.cap = pd.read_csv(dir + "meta_data_cap.csv")
        self.root_dir = root_dir
        self.transform = transform
        self.all_images = []
        self.patientid_to_label = patientid_to_label
        self.invalid_patients = invalid_patients

        self.images = defaultdict(list)
        self.invalid_patients = defaultdict(lambda: False)
        self.patientid_to_label = {}

        def resizeAndPad(img, size, padColor=0):

          h, w = img.shape[:2]
          sh, sw = size

          # interpolation method
          if h > sh or w > sw: # shrinking image
              interp = cv2.INTER_AREA
          else: # stretching image
              interp = cv2.INTER_CUBIC

          # aspect ratio of image
          aspect = w/h  # if on Python 2, you might need to cast as a float: float(w)/h

          # compute scaling and pad sizing
          if aspect > 1: # horizontal image
              new_w = sw
              new_h = np.round(new_w/aspect).astype(int)
              pad_vert = (sh-new_h)/2
              pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
              pad_left, pad_right = 0, 0
          elif aspect < 1: # vertical image
              new_h = sh
              new_w = np.round(new_h*aspect).astype(int)
              pad_horz = (sw-new_w)/2
              pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
              pad_top, pad_bot = 0, 0
          else: # square image
              new_h, new_w = sh, sw
              pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

          # scale and pad
          scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
          scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

          return scaled_img

        def process_data (label, dir):
          invalid = 0
          for index, row in label.iterrows():
            if not index % 100:
              print(dir[0], index)
            img_array = cv2.imread(root_dir + "/" + dir + "/" + row["File name"])
            if not is_grayscale(img_array):
              invalid+=1
              print("skipped/invalid: ", invalid)
              self.invalid_patients[row["Patient ID"]] = True
              continue
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
            _, thresholded = cv2.threshold(img_array, 0, 255, cv2.THRESH_OTSU)
            bbox = cv2.boundingRect(thresholded)
            x, y, w, h = bbox

            img_array = img_array[y:y+h, x:x+w]
            img_array = resizeAndPad(img_array, (512,512))

            self.images[row["Patient ID"]].append(img_array)
            self.patientid_to_label[row["Patient ID"]] = row["Diagnosis"]

        if not data:
          process_data(self.normal, "1NonCOVID")
          process_data(self.covid, "2COVID")
          process_data(self.cap, "3CAP")

          for patient_id, images in self.images.items():
            if len(images) >= 3 and not self.invalid_patients[patient_id]:
              self.all_images.append((images, self.patientid_to_label[patient_id], patient_id))

          random.shuffle(self.all_images)

        else:
          self.all_images = data


    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        images = self.all_images[idx][0][:3]
        label = self.all_images[idx][1]
        patient_id = self.all_images[idx][2]

        images = np.array(images)
        
        if self.transform:
            images = self.transform(images)

        sample = {'images': images, 'label': label, 'patient' : patient_id}

        return sample



In [10]:
def resizeAndPad(img, size, padColor=0):

  h, w = img.shape[:2]
  sh, sw = size

  # interpolation method
  if h > sh or w > sw: # shrinking image
      interp = cv2.INTER_AREA
  else: # stretching image
      interp = cv2.INTER_CUBIC

  # aspect ratio of image
  aspect = w/h  # if on Python 2, you might need to cast as a float: float(w)/h

  # compute scaling and pad sizing
  if aspect > 1: # horizontal image
      new_w = sw
      new_h = np.round(new_w/aspect).astype(int)
      pad_vert = (sh-new_h)/2
      pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
      pad_left, pad_right = 0, 0
  elif aspect < 1: # vertical image
      new_h = sh
      new_w = np.round(new_h*aspect).astype(int)
      pad_horz = (sw-new_w)/2
      pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
      pad_top, pad_bot = 0, 0
  else: # square image
      new_h, new_w = sh, sw
      pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

  # set pad color
  if len(img.shape) == 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided
      padColor = [padColor]*3

  # scale and pad
  scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
  scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

  return scaled_img

In [11]:
with open(dir+'all_images.pkl', 'rb') as file:

    all_images = pickle.load(file)

In [12]:
mean = 7.847408137450277e-06 
std = 7.222772184630013e-06

data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

CT_dataset = CTScanDataset(root_dir = datapath, data=all_images)

In [13]:
dataloader = DataLoader(CT_dataset, batch_size=32, num_workers=0)
def show_landmarks_batch(sample_batched):

    for i_batch, sample_batched in enumerate(dataloader):
        print(i_batch, sample_batched['image'].size(),
            sample_batched['landmarks'].size())

        # observe 4th batch and stop.
        if i_batch == 3:
            plt.figure()
            show_landmarks_batch(sample_batched)
            plt.axis('off')
            plt.ioff()
            plt.show()
            break

In [14]:
dataloader

In [15]:
def getImage(i, l):
    return np.array(CT_dataset.__getitem__(i)['images'][l])

def showPatientSample(i, l):
    img = getImage(i,l)
    plt.imshow(img, cmap='gray')
    print(img.shape)
    print(CT_dataset.__getitem__(i)['label'])
    print(CT_dataset.__getitem__(i)['patient'])

In [16]:
c=0
for batch in dataloader:
    print(batch)
    

    

{'images': tensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]],


        [[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,

In [17]:
from torchvision import transforms

def get_mean_std(loader):
    # Compute the mean and standard deviation of all pixels in the dataset
    num_pixels = 0
    mean = 0.0
    std = 0.0
    for batch in loader:
        images = np.array(batch["images"])
        num_images, batch_size, height, width = images.shape
        images = np.reshape(images, (batch_size, num_images, height, width))
        print(images.shape)
        num_pixels += batch_size * height * width
        mean += images.mean(axis=(0, 1, 2, 3)).sum()
        std += images.std(axis=(0, 1, 2, 3)).sum()

    mean /= num_pixels
    std /= num_pixels

    return mean, std

mean, std = get_mean_std(dataloader)

'''data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])'''


(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 32, 512, 512)
(3, 8, 512, 512)


'data_transforms = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize(mean=mean, std=std)\n])'

In [2]:
from argparse import Namespace

args = Namespace(batch_size=32,
epochs=400,
accum_iter=1,
model='mae_vit_base_patch16',
input_size=512,
mask_ratio=0.75,
norm_pix_loss=False,
weight_decay=0.05,
lr=None,
blr=0.001,
min_lr=0.0,
warmup_epochs=40,
data_path='G:/My Drive/CTdata/transformedData',
output_dir='./output_dir',
log_dir='./output_dir',
device='cuda',
seed=0,
resume='', # ***
start_epoch=0,
num_workers=10,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://',
distributed=False)

In [3]:
import models_mae
model = models_mae.__dict__[args.model](norm_pix_loss=args.norm_pix_loss)

In [20]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epochs=10

for epoch in range(epochs):
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        inputs = batch['images']
        labels = batch['label']

        num_images, batch_size, height, width = inputs.shape
        inputs.reshape((batch_size, num_images, height, width))

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(CT_dataset)
    print(f"Epoch {epoch+1} loss: {epoch_loss:.4f}")

AssertionError: Input height (512) doesn't match model (224).

In [ ]:
for batch in dataloader:
    print(np.shape(batch['images']))

(3, 512, 512)


ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
for batch in CT_dataset:
  images = batch["images"]
  if np.array(images).shape == (3,):
    print(images)

In [ ]:
shapes =[]

for images, label, patient_id in all_images:
  for image in images:
    shp = np.array(image).shape
    shapes.append(shp)
    if shp == (1276, 1858):
      print(label, patient_id)

In [ ]:
covid[covid['Patient ID'] == "217"].index.values

In [ ]:
img = np.asarray(Image.open(datapath+'/2COVID/' + "15_Cohen_16630_2_2.png"))
imgplot = plt.imshow(img)

In [ ]:
img = np.asarray(Image.open(datapath+'/2COVID/' + "15_Cohen_16630_2_1.png"))
imgplot = plt.imshow(img)